In [1]:
# ----------------------------
# 1. Install Dependencies
# ----------------------------
!pip install -q transformers datasets pandas torch
!pip install -q sentencepiece


# ----------------------------
# 2. Import Modules
# ----------------------------
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import torch
import os

# ----------------------------
# 3. Load and Clean Data
# ----------------------------
df = pd.read_csv("generator_data.csv")
print("Original length:", len(df))
print("Missing values:\n", df.isna().sum())

# Rename columns for clarity
df = df.rename(columns={"target_dna": "input_text", "grna_seq": "target_text"})

# Drop missing or duplicate rows
df = df.dropna(subset=["input_text", "target_text"])
df = df.drop_duplicates(subset=["input_text", "target_text"])

# Save cleaned data
df.to_csv("generator_data_clean.csv", index=False)

# ----------------------------
# 4. Load Dataset
# ----------------------------
dataset = load_dataset("csv", data_files="generator_data_clean.csv")
print("Example:", dataset["train"][0])

# ----------------------------
# 5. Load Tokenizer and Model
# ----------------------------
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ----------------------------
# 6. Preprocessing Function
# ----------------------------
def preprocess(example):
    input_seq = example["input_text"].strip()
    target_seq = example["target_text"].strip()
    model_input = tokenizer("generate gRNA: " + input_seq, padding="max_length", truncation=True, max_length=64)
    label = tokenizer(text_target=target_seq, padding="max_length", truncation=True, max_length=32)
    model_input["labels"] = label["input_ids"]
    return model_input

# Tokenize
tokenized_dataset = dataset.map(preprocess, batched=False)
tokenized_dataset.set_format("torch")

# ----------------------------
# 7. Training Arguments
# ----------------------------
training_args = TrainingArguments(
    output_dir="./results",               # directory to save
    per_device_train_batch_size=16,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="no",                   # disable saving to avoid disk quota issues
    fp16=torch.cuda.is_available(),       # use mixed precision on GPU
    report_to="none",                     # disable wandb/tensorboard logs
)

# ----------------------------
# 8. Define Trainer
# ----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

# ----------------------------
# 9. Temporary DTensor Patch
# ----------------------------
import builtins
class DTensor: pass
builtins.DTensor = DTensor

# ----------------------------
# 10. Train Model
# ----------------------------
trainer.train()


2025-06-04 05:50:44.886552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749016244.907190     920 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749016244.913487     920 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 05:50:44.937419: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Original length: 19999
Missing values:
 input_text     31
target_text     0
dtype: int64


Generating train split: 0 examples [00:00, ? examples/s]

Example: {'input_text': 'CGGCGCTGGTGCCCAGGACGAGGATGGAGATT', 'target_text': 'CGGCGCTGGTGCCCAGGACGAGGATGGAGATT'}


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/15379 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,9.033200
20,4.301900
30,2.430000
40,2.039100
50,1.792600
60,1.613500
70,1.481800
80,1.384800
90,1.238200
100,1.150600


TrainOutput(global_step=2886, training_loss=0.8957940663055505, metrics={'train_runtime': 254.7799, 'train_samples_per_second': 181.086, 'train_steps_per_second': 11.327, 'total_flos': 780533086814208.0, 'train_loss': 0.8957940663055505, 'epoch': 3.0})